In [ ]:
!pip install wikipedia-api -q

In [ ]:
import time
import json
import string

import requests
import wikipediaapi

EMAIL = ''

wiki = wikipediaapi.Wikipedia(
  language='en',
  extract_format=wikipediaapi.ExtractFormat.WIKI
)

def random_page():
  url = 'https://en.wikipedia.org/api/rest_v1/page/random/title'
  title = requests.get(url).json()['items'][0]['title']

  main_page = wiki.page(title)
  related_pages = get_related(title)
  sentences = []
  for page in [main_page, *related_pages]:
    sentences.extend(parse_page(page))
    time.sleep(0.2)  # one of these is redundant but I don't know which
  
  return title, sentences

def get_related(title):
  resp = requests.get(f'https://en.wikipedia.org/api/rest_v1/page/related/{title}').json()
  titles = []
  for page in resp['pages']:
    titles.append(page['title'])
  pages = []
  for title in titles:
    pages.append(wiki.page(title))
    time.sleep(0.2)  # one of these is redundant but I don't know which
  return pages

def parse_page(page):
  sections = page.text.splitlines()
  sentences = []
  for section in sections:
    new_sentences = section.split('. ')
    clean_sentences = []
    # make a translation that deletes all punctuation so that I can run it on
    # all the sentences and get rid of ones that are just punctuation
    # with some whitespace
    trans = ''.maketrans('','',string.punctuation)
    for sentence in new_sentences:
      stripped = sentence.translate(trans).strip()
      if stripped == '':
        continue
      # I want the version that has punctuation
      clean_sentences.append(sentence)
    sentences.extend(clean_sentences)

  return sentences 

def create_template():
  template_url = 'https://api.berri.ai/create_template'

  template_config = {"advanced": {
    "intent": "qa_doc",
    "search": "default"
    }
  }
  resp = requests.post(template_url, data={"app_config": json.dumps(template_config)})
  return resp.json()

def create_instance(template_id):
  title, sentences = random_page()
  create_url = 'https://api.berri.ai/create_app'
  config = {
      'template_id': template_id,
      'user_email':EMAIL,
      'data_source': json.dumps(sentences),
      'prompt': f'You help people learn more about {title} using information from Wikipedia! You start each conversation with a fun fact about {title}!',
  }
  resp = requests.post(url=create_url, data=config)
  return resp.json()

def delete_instance(instance_id):
  delete_url = 'https://api.berri.ai/delete_instance'
  params = {
      'user_email': EMAIL,
      'instance_id': instance_id
  }
  r = requests.post(url=delete_url, params=params)
  return r.json()

In [ ]:
template_id = create_template()['template_id']
app_json = create_instance(template_id)
app_id = app_json['instance_id']
app_website = app_json['website_endpoint']
print(f'created app {app_id}\nAccess it here:\n{app_website}')

created app 7e6e9055-1099-437a-8948-0ec04a9400c8
Access it here:
chat.berri.ai/aHR0cHM6Ly9zdG9yZXF1ZXJ5YWJoaTItYXlsdS56ZWV0LWJlcnJpLnplZXQuYXBwL2JlcnJpX3F1ZXJ5P3Byb2pfcGF0aD1pbmRleGVzL2ZhY2FkZUBkdWNrLmNvbS83ZTZlOTA1NS0xMDk5LTQzN2EtODk0OC0wZWMwNGE5NDAwYzgmcHJval9uYW1lPWRhdGFfbGlzdCZhZ2VudF90eXBlPXNpbXBsZV9zdXBwb3J0JnF1ZXJ5PQ==


In [ ]:
# delete_instance(app_id)

{'message': 'Instance 7e6e9055-1099-437a-8948-0ec04a9400c8 deleted successfully',
 'status': 'success'}